
# Using UV ZO Unit Model


## Step 1: Import the necessary functions

In [6]:
from pyomo.environ import ConcreteModel, Suffix, units as pyunits
from idaes.core import FlowsheetBlock
from watertap.core.zero_order_properties import WaterParameterBlock
from watertap.core.wt_database import Database
from watertap.unit_models.zero_order.uv_zo import UVZO
from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.environ import ConcreteModel, assert_optimal_termination
from pyomo.util.check_units import assert_units_consistent
from watertap.core.solvers import get_solver

## Step 2: Create the ConcreteModel and FlowsheetBlock
Create the flowsheet by attaching the property package and building the custom unit model.

In [11]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
# Attach property package
m.fs.db = Database('../watertap/data/techno_economic')
m.fs.properties = WaterParameterBlock(database=m.fs.db)
# Build the unit model
m.fs.unit = UVZO(property_package=m.fs.properties, database=m.fs.db)

# Display model
print(m.fs.unit.display())

Block fs.unit

  Variables:
    _flow_mass_comp_inlet_ref : Size=12, Index=fs._time*fs.properties.component_list, ReferenceTo=fs.unit.properties_in[...].component('flow_mass_comp')[...]
        Key                : Lower : Value : Upper : Fixed : Stale : Domain
              (0.0, 'H2O') :     0 :     1 :  None : False : False : PositiveReals
            (0.0, 'boron') :     0 :     1 :  None : False : False : PositiveReals
          (0.0, 'bromide') :     0 :     1 :  None : False : False : PositiveReals
          (0.0, 'calcium') :     0 :     1 :  None : False : False : PositiveReals
         (0.0, 'chloride') :     0 :     1 :  None : False : False : PositiveReals
        (0.0, 'magnesium') :     0 :     1 :  None : False : False : PositiveReals
        (0.0, 'potassium') :     0 :     1 :  None : False : False : PositiveReals
           (0.0, 'sodium') :     0 :     1 :  None : False : False : PositiveReals
        (0.0, 'strontium') :     0 :     1 :  None : False : False : Posit

## Step 3: Set the operating conditions of the unit model
Specify the feed conditions and unit model variables such that the degrees of freedom are zero. Default scaling should also be set for the flow rate to ensure the model is well-scaled.

In [3]:

print("DOF before specifying:", degrees_of_freedom(m.fs))

# --- load & FIX zero-order params from DB (removals, energy intensity, recovery, etc.) ---
# use_default_removal=True will fix any missing removals to a small default instead of leaving them free
m.fs.unit.load_parameters_from_database(use_default_removal=True)

# --- specify inlet mass flows (one per component) ---
pin = m.fs.unit.properties_in[0]

pin.flow_mass_comp["H2O"].fix(1.000)  # 1 kg/s water
pin.flow_mass_comp["tss"].fix(0.500)  # 0.1 kg/s TSS

t0 = 0.0
pint  = m.fs.unit.properties_in[t0].flow_mass_comp
pout = m.fs.unit.properties_treated[t0].flow_mass_comp

print("inlet tss =", float(pint["tss"].value))
print("treated tss =", float(pout["tss"].value))

# list all components + values (so we see what's actually in the state block)
comps = list(m.fs.properties.component_list)
vals_in  = {c: float(pint[c].value)  for c in comps}
vals_out = {c: float(pout[c].value) for c in comps}
print("components:", comps)
print("in:",  vals_in)
print("out:", vals_out)


# for solutes
for s in m.fs.properties.solute_set:
    pin.flow_mass_comp[s].fix(1e-6)

# --- UV operating inputs ---
m.fs.unit.uv_transmittance_in[0].fix(0.85)  # fraction (0–1)
m.fs.unit.uv_reduced_equivalent_dose[0].fix(40.0 * pyunits.mJ/pyunits.cm**2)  # typical 40 mJ/cm^2

print("DOF after spec:", degrees_of_freedom(m.fs))  # should be 0


DOF before specifying: 24
inlet tss = 0.5
treated tss = 1.0
components: ['H2O', 'boron', 'bromide', 'calcium', 'chloride', 'magnesium', 'potassium', 'sodium', 'strontium', 'sulfate', 'tds', 'tss']
in: {'H2O': 1.0, 'boron': 1.0, 'bromide': 1.0, 'calcium': 1.0, 'chloride': 1.0, 'magnesium': 1.0, 'potassium': 1.0, 'sodium': 1.0, 'strontium': 1.0, 'sulfate': 1.0, 'tds': 1.0, 'tss': 0.5}
out: {'H2O': 1.0, 'boron': 1.0, 'bromide': 1.0, 'calcium': 1.0, 'chloride': 1.0, 'magnesium': 1.0, 'potassium': 1.0, 'sodium': 1.0, 'strontium': 1.0, 'sulfate': 1.0, 'tds': 1.0, 'tss': 1.0}
DOF after spec: 0


## Step 4: Solve the flowsheet and display results

In [4]:
# Check that units are consistent
assert_units_consistent(m) 

# Check that the degrees of freedom are what we expect
assert (
    degrees_of_freedom(m) == 0
)  

solver = get_solver()
results = solver.solve(m, tee=False)

# Check that the solver finds an optimal solution
assert_optimal_termination(results)

# Display results
print("second display")
m.fs.unit.display()

second display
Block fs.unit

  Variables:
    _flow_mass_comp_inlet_ref : Size=12, Index=fs._time*fs.properties.component_list, ReferenceTo=fs.unit.properties_in[...].component('flow_mass_comp')[...]
        Key                : Lower : Value : Upper : Fixed : Stale : Domain
              (0.0, 'H2O') :     0 :   1.0 :  None :  True :  True : PositiveReals
            (0.0, 'boron') :     0 : 1e-06 :  None :  True :  True : PositiveReals
          (0.0, 'bromide') :     0 : 1e-06 :  None :  True :  True : PositiveReals
          (0.0, 'calcium') :     0 : 1e-06 :  None :  True :  True : PositiveReals
         (0.0, 'chloride') :     0 : 1e-06 :  None :  True :  True : PositiveReals
        (0.0, 'magnesium') :     0 : 1e-06 :  None :  True :  True : PositiveReals
        (0.0, 'potassium') :     0 : 1e-06 :  None :  True :  True : PositiveReals
           (0.0, 'sodium') :     0 : 1e-06 :  None :  True :  True : PositiveReals
        (0.0, 'strontium') :     0 : 1e-06 :  None :  True 

In [5]:
print("treated tss =", float(pout["tss"].value))

treated tss = 9.999999999999972e-07
